# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import tweepy
from tweepy import OAuthHandler
import json

import requests
import time
import config
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
tweets_df = pd.read_csv('twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
path_to_file = 'image_predictions.tsv'
path = Path(path_to_file)

if not path.is_file():

    images_data = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
    images_data.close()


    with open('image_predictions.tsv', 'wb') as f:
        f.write(images_data.content)
    f.close()

image_preds = pd.read_csv('image_predictions.tsv', sep='\t')
image_preds.head()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
auth = OAuthHandler(config.CONSUMER_KEY, config.CONSUMER_SECRET)
auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

path_to_file = 'tweet_json.txt'
path = Path(path_to_file)

if not path.is_file():

    tweet_ids = tweets_df.tweet_id.values 

    count = 0
    fails_dict = {}
    start = time.time()
    with open('tweet_json.txt', 'w') as outfile:
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.errors.HTTPException as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
    print(fails_dict)
    print('\n')
    print('time elapsed: ', time.time()-start)

In [ ]:
retweets_df = pd.read_json('tweet_json.txt', lines=True)
retweets_df = retweets_df[['created_at', 'id', 'retweet_count', 'favorite_count']]

## Assessing Data

#### tweets_df

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.info()

In [ ]:
tweets_df.describe()

In [ ]:
tweets_df.sample(7)

In [ ]:
tweets_df.rating_denominator.value_counts()

In [ ]:
tweets_df.name.value_counts()[:15]

#### image_preds

In [ ]:
image_preds.head()

In [ ]:
image_preds.info()

In [ ]:
image_preds.describe()

In [ ]:
image_preds.sample(10)

In [ ]:
image_preds[(image_preds.p1_dog != True) & (image_preds.p2_dog != True) & (image_preds.p3_dog != True)]
# records of invalid dog images

In [ ]:
retweets_df.info()

In [ ]:
retweets_df.sample(5)

### Quality issues
1. `tweets_df`: `timestamp` column is a pandas object datatype instead of a datetime datatype

2. `tweets_df`: retweets rows and columns found in this tweets dataset

2. `tweets_df`: Stop words like *a, the, ..* found in `name`

3. `tweets_df`: Nulls represented as *None* in `name`

4. `tweets_df`: Nulls represented as *None* in `doggo, floofer, pupper & puppo`

5. `tweets_df`: `rating_denominator` has incorrect values which include numbers < 10 (also the number 11 is incorrect)

6. `tweets_df`: The `text` column serves as redundant data

7. `image_preds`: Inconsistent capitilization of names in `p1, p2 & p3` (some name of dog types begin with lower case).

8. `image_preds`: columns `p1_dog, p2_dog, p3_dog` have records in which all are false, making them invalid dog type images.

9. `retweets_df`: `id` column instead of *twitter_id* column

### Tidiness issues
1. Dog stages represented in 4 columns instead of 1 in `tweets_df`

2. Only most confident predicted image column is needed in `image_preds`

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
tweets_df_copy = tweets_df.copy()
retweet_df_copy = retweets_df.copy()
image_preds_copy = image_preds.copy()


### Issue #1:

#### Define:
Change timestamp to datatime datatype using pandas `to_datetime` method

#### Code

In [ ]:
tweets_df_copy.timestamp = pd.to_datetime(tweets_df_copy.timestamp)
#changing timestamp to datetime datatype

#### Test

In [ ]:
tweets_df_copy.info()

### Issue #2:

#### Define

Delete rows and columns with retweet information using pandas `iloc` and `drop` methods

#### Code

In [ ]:
tweets_df_copy = tweets_df_copy[tweets_df_copy.retweeted_status_id.isnull()]
tweets_df_copy = tweets_df_copy[tweets_df_copy.in_reply_to_status_id.isnull()]
tweets_df_copy.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

# dropping off retweet records and columns

#### Test

In [ ]:
tweets_df_copy.info()

### Issue #3:

#### Define:


Replace stop words with `np.nan`

#### Code

In [ ]:
def replace_names(tweets):
    """
    Replaces stop words with NaN
    
    Arguments:
        tweets: a dataframe

    Returns:
        null values inplace of stop words in the name column
    """

    if tweets['name'][0].islower():
        new_name = np.nan
        return new_name
    else:
        return tweets['name']

tweets_df_copy.name = tweets_df_copy.apply(replace_names, axis=1)

#### Test

In [ ]:
tweets_df_copy.name.value_counts()[:15]

### Issue #4:

#### Define

replace None with `np.nan` in `name` column

#### Code

In [ ]:
tweets_df_copy.name.replace('None', np.nan, inplace=True)
# replaces None with null value

#### Test

In [ ]:
tweets_df_copy.name.value_counts()

### Issue #5:

#### Define

Replace Nulls with `np.nan` in columns `doggo, floofer, pupper & puppo`

#### Code

In [ ]:
tweets_df_copy.doggo.replace('None', np.nan, inplace=True)
tweets_df_copy.floofer.replace('None', np.nan, inplace=True)
tweets_df_copy.pupper.replace('None', np.nan, inplace=True)
tweets_df_copy.puppo.replace('None', np.nan, inplace=True)

#### Test

In [ ]:
sum(tweets_df_copy.doggo.values == 'None')

### Issue #6:

#### Define

Replace values less than 10 and which are 11 in the `rating_denominator` column with 10

#### Code

In [ ]:
def replace_values(tweets):

    """ 
    Replaces values less than 10 with 10

    Arguments:
        tweets: a dataframe

    Returns:
        10 for values less than 10 in the rating_denominator column
    """

    if tweets['rating_denominator'] < 10:
        new_value = 10
        return new_value
    else:
        return tweets['rating_denominator']

tweets_df_copy['rating_denominator'] = tweets_df_copy.apply(replace_values, axis=1)

In [ ]:
tweets_df_copy = tweets_df_copy[tweets_df_copy.rating_denominator !=11]

#### Test

In [ ]:
tweets_df_copy.info()

In [ ]:
tweets_df_copy[(tweets_df_copy.rating_denominator < 10) & (tweets_df_copy.rating_denominator == 11)]

### Issue #7:

#### Define

Drop the _text_ column using pandas `drop` method

#### Code

In [ ]:
tweets_df_copy.drop('text', axis=1, inplace=True)
# drops off text column

#### Test

In [ ]:
tweets_df_copy.info()

### Issue #8:

#### Define

Make all all values of `p1, p2 & p3` to be lowercase

#### Code

In [ ]:
image_preds_copy[['p1', 'p2', 'p3']] = image_preds_copy[['p1', 'p2', 'p3']].apply(lambda x: x.str.lower())
# transform all dog breeds names to lowercase

#### Test

In [ ]:
image_preds_copy.head()

### Issue #9:

#### Define

Factor out rows which have all `p1_dog, p2_dog & p3_dog` values as `False`

#### Code

In [ ]:
image_preds_copy = image_preds_copy[~((image_preds_copy.p1_dog == False) & (image_preds_copy.p2_dog == False) &(image_preds_copy.p3_dog == False))]
#returns only true dog image records

#### Test

In [ ]:
image_preds_copy[(image_preds_copy.p1_dog == False) & (image_preds_copy.p2_dog == False) &(image_preds_copy.p3_dog == False)]

### Issue #10:

#### Define

Rename `id` column to `tweet_id` using pandas' `rename` method for **retweets_df**

#### Code

In [ ]:
retweet_df_copy.rename(columns={'id': 'tweet_id'}, inplace=True)

In [ ]:
retweet_df_copy.drop(['created_at'], axis=1, inplace=True)

#### Test

In [ ]:
retweet_df_copy.info()

### Issue #11(Tidiness):

#### Define

Drop columns `doggo, floofer, pupper & puppo` and make new column, `dog_stage` to represent values of all 4 dropped columns

#### Code

In [ ]:
df1 = tweets_df_copy.copy()
df2 = tweets_df_copy.copy()
# makes two copies of the tweets_df dataset

df1 = df1[(df1['doggo'].isnull()) & (df1['floofer'].isnull()) & (df1['pupper'].isnull()) & (df1['puppo'].isnull())]
df1.drop(['doggo','floofer','pupper','puppo'], axis=1, inplace=True)
# df1 to contain no known dog stage

dog_stage = np.empty(len(df1))
dog_stage[:] = np.nan
df1['dog_stage'] = dog_stage
# creating a df1 column of an empty dog stage


df2 = df2[(df2['doggo'].notnull()) | (df2['floofer'].notnull()) | (df2['pupper'].notnull()) | (df2['puppo'].notnull())]
# df2 to contain at least one known dog stage

df2 = pd.melt(df2, id_vars=['tweet_id', 'timestamp','source',
                        'expanded_urls','rating_numerator',
                        'rating_denominator','name'], value_name='dog_stage')
# melting the 4 dog stages (doggo, floofer, pupper & puppo) into dog_stage

df2.drop('variable', axis=1, inplace=True)
df2 = df2[df2['dog_stage'].notnull()]
df2 = df2[~((df2.tweet_id.duplicated()) & (df2.dog_stage.notnull()))]

tweets_df_copy = pd.concat([df1, df2], ignore_index=True)
# concating df1 and df2 back to give tweets_df_copy


#### Test

In [ ]:
tweets_df_copy.sample(5)

In [ ]:
tweets_df_copy.info()

### Issue #12(Tidiness):

#### Define 

Return new column name: `dog_type` for `p1_conf or p2_conf or p3_conf` with the highest value 

#### Code

In [ ]:
def right_dog(image):

    """ 
    Compares three confidence values

    Arguments:
        image: a dataframe

    Returns:
        dog_type: column of dog breed names with highest predicted dog breed name
    """

    if image['p1_conf'] > image['p2_conf'] > image['p3_conf'] and image['p1_dog'] == True:
        return image['p1']
    elif image['p2_conf'] > image['p3_conf'] and image['p2_dog'] == True:
        return image['p2']
    else:
        return image['p3']

image_preds_copy['dog_type'] = image_preds_copy.apply(right_dog, axis=1)

In [ ]:
image_preds_copy.drop(['p1', 'p1_conf', 'p1_dog','p2', 'p2_conf', 'p2_dog','p3', 'p3_conf', 'p3_dog'], axis=1, inplace=True)

#### Test

In [ ]:
image_preds_copy.sample(5)

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
twitter_df_clean = pd.merge(tweets_df_copy, retweet_df_copy, on=['tweet_id'], how='left')
twitter_df_clean = pd.merge(twitter_df_clean, image_preds_copy, on=['tweet_id'])
twitter_df_clean.info()

In [ ]:
twitter_df_clean.head()

In [ ]:
twitter_df_clean.to_csv('twitter_archive_master.csv', index=False)
# saving the combined dataset as a csv document

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

In [ ]:
twitter_df_clean.dog_type.value_counts()[:10]

In [ ]:
twitter_df_clean.groupby('rating_numerator')['favorite_count'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('rating_numerator')['retweet_count'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('rating_numerator')['favorite_count'].count().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('dog_type')['retweet_count'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('dog_type')['favorite_count'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('dog_stage')['rating_numerator'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('dog_stage')['retweet_count'].mean().sort_values(ascending=False)

In [ ]:
twitter_df_clean.groupby('dog_stage')['favorite_count'].mean().sort_values(ascending=False)

### Insights:
1. The Golden retriever is the most common dog in the WeRateDog archive

2. The Bedlington terrier is the most liked as well as the most retweeted dog type. Others which make this list include: French bulldog and Afghan hound

3. Floofer dogs get better average numerator ratings than others

### Visualization

#### Plot function

In [ ]:
def plot_bar(data, color, label_x, label_y, title):

    """ 
    Plots a bar plot between two correlated variables

    Arguments:
        data: a dictionary
        color: a string
        label_X: a string
        label_y: a string

    Returns:
        a bar plot
    """

    height = list(data.keys())
    values = list(data.values())
    
    fig = plt.figure(figsize=(12,8))
    plt.bar(height, values, color=color)
    plt.xlabel(label_x, fontsize=14)
    plt.ylabel(label_y, fontsize=14)
    plt.title(title, fontsize=18)
    plt.tick_params(left=False, bottom=False)

In [ ]:
twitter_df_clean.plot.scatter(x='rating_numerator', y='favorite_count')

In [ ]:
floofer_df = twitter_df_clean.query('dog_stage == "floofer"')['rating_numerator'].mean()
doggo_df = twitter_df_clean.query('dog_stage == "doggo"')['rating_numerator'].mean()
puppo_df = twitter_df_clean.query('dog_stage == "puppo"')['rating_numerator'].mean()
pupper_df = twitter_df_clean.query('dog_stage == "pupper"')['rating_numerator'].mean()

data = {'floofer':floofer_df, 'doggo':doggo_df, 'puppo':puppo_df, 'pupper':pupper_df}

plot_bar(data, color='maroon', label_x='Dog_stage', label_y='Average rating', title='Average Rating of Dog stages')


In [ ]:
data = {'floofer':floofer_df, 'doggo':doggo_df, 'puppo':puppo_df, 'pupper':pupper_df}
height = list(data.keys())
values = list(data.values())

fig = plt.figure(figsize=(12,8))
plt.plot(height, values)
plt.xlabel('Dog_stage')
plt.ylabel('Average rating')
plt.title('Line graph of average rating of dog stages')

In [ ]:
favourite_floffer = twitter_df_clean.query('dog_stage == "floofer"')['favorite_count'].mean()
favourite_doggo = twitter_df_clean.query('dog_stage == "doggo"')['favorite_count'].mean()
favourite_pupppo = twitter_df_clean.query('dog_stage == "puppo"')['favorite_count'].mean()
favourite_pupper = twitter_df_clean.query('dog_stage == "pupper"')['favorite_count'].mean()

data = {'floofer':favourite_floffer, 'doggo':favourite_doggo, 'puppo':favourite_pupppo, 'pupper':favourite_pupper}

plot_bar(data, color='green', label_x='dog stage', label_y='favorite count', title='Average Favorite count of Dog Stages')

In [ ]:
retweet_floffer = twitter_df_clean.query('dog_stage == "floofer"')['retweet_count'].mean()
retweet_doggo = twitter_df_clean.query('dog_stage == "doggo"')['retweet_count'].mean()
retweet_pupppo = twitter_df_clean.query('dog_stage == "puppo"')['retweet_count'].mean()
retweet_pupper = twitter_df_clean.query('dog_stage == "pupper"')['retweet_count'].mean()

data = {'floofer':retweet_floffer, 'doggo':retweet_doggo, 'puppo':retweet_pupppo, 'pupper':retweet_pupper}

plot_bar(data, color='#00008b', label_x='dog stage', label_y='retweet count', title='Average Retweet count of Dog Stages')